In [1]:
# https://nbconvert.readthedocs.io/en/latest/removing_cells.html
# use these magic spells to update your classes methods on-the-fly as you edit them:
%reload_ext autoreload
%autoreload 2
from pprint import pprint
from IPython.core.display import display, HTML, Markdown
import ipywidgets as widgets
# %run includeme.ipynb # include a notebook from this same directory
display(HTML("<style>.container { width:100% !important; }</style>"))

/tmp/ipykernel_6230/1348678174.py:6: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML, Markdown


The mpeg-vcm working group defines a specially prepared custom datasets (subset of OpenImageV6) for evaluating the performance of your deep-learning de/compression algorithm.

The tricky part is importing all that data into fiftyone.  Once we have done that, we can use the CLI tools to evaluate the de/compression model with the mpeg-vcm defined pipeline, i.e.:
```
mpeg-vcm custom dataset --> compression and decompression --> Detectron2 predictor --> mAP
```

The CLI tools have a subcommand ``mpeg-vcm-auto-import`` that downloads necessary images from the open images dataset and prepares the annotations according to mpeg-vcm working group specifications, so the only thing you need to do to get started, is simply to type
```
compressai-vision mpeg-vcm-auto-import
```
After running that "wizard", you should have the mpeg-vcm datasets registered into fiftyone (``mpeg-vcm-detection`` etc. datasets):

In [2]:
!compressai-vision list

importing fiftyone
fiftyone imported

datasets currently registered into fiftyone
name, length, first sample path
mpeg-vcm-detection, 5000, /home/sampsa/fiftyone/mpeg-vcm-detection/data
mpeg-vcm-detection-dummy, 1, /home/sampsa/fiftyone/mpeg-vcm-detection/data
mpeg-vcm-segmentation, 5000, /home/sampsa/fiftyone/mpeg-vcm-segmentation/data
open-images-v6-validation, 8189, /home/sampsa/fiftyone/open-images-v6/validation/data
quickstart, 200, /home/sampsa/fiftyone/quickstart/data
